# Imports

In [ ]:
# remove warning message
%load_ext autoreload
%autoreload 2

# required library
from flask import Flask, request, redirect, url_for, flash, jsonify
from flasgger import Swagger, swag_from # documentação da api em swagger
import flasgger
import pandas as pd
import numpy as np
import seaborn as sns
import os
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Process and Predict

In [ ]:
def process_and_predict(file):
    im = Image.open(file)
    width, height = im.size
    if width == height:
        im = im.resize((200,200), Image.ANTIALIAS)
    else:
        if width > height:
            left = width/2 - height/2
            right = width/2 + height/2
            top = 0
            bottom = height
            im = im.crop((left,top,right,bottom))
            im = im.resize((200,200), Image.ANTIALIAS)
        else:
            left = 0
            right = width
            top = 0
            bottom = width
            im = im.crop((left,top,right,bottom))
            im = im.resize((200,200), Image.ANTIALIAS)
            
    ar = np.asarray(im)
    ar = ar.astype('float32')
    ar /= 255.0
    ar = ar.reshape(-1, 200, 200, 3)
    
    age = agemodel.predict(ar)
    gender = np.round(genmodel.predict(ar))
    if gender == 0:
        gender = 'Masculino'
    elif gender == 1:
        gender = 'Feminino'
    
    print('Idade:', int(age), '\nGenero:', gender)
    return int(age), gender

# Load Model

In [ ]:
genmodel = keras.models.load_model('models/genero_modelo.h5')
agemodel = keras.models.load_model('models/idade_modelo.h5')

process_and_predict('assets/thisperson4.jpg')

# WebService Definitons

In [ ]:
UPLOAD_FOLDER = 'assets/'
ALLOWED_EXTENSIONS = set(['tif', 'png', 'jpg', 'jpeg', 'gif','undefined'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS

#(criar o servidor) flask
app = Flask(__name__)
Swagger(app)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# WebService Routes

In [ ]:
@app.route('/api/predict', methods=['POST'])
def m_upload():
    """
    Webservice endpoint that given an user image returns the predicted age and gender
    ---
    consumes:
       - multipart/form-data
    parameters:
       - name: image
         in: formData
         type: file
         required: true
    responses:
        200:
            description : predicted age and gender
    """
    
    # check if the post request has the file part
    if 'image' not in request.files:
        return jsonify({'error':'No posted image. Should be attribute named image.'})
    file = request.files['image']
    # if user does not select file, browser also
    # submit a empty part without filename
    if file.filename == '':
        return jsonify({'error':'Empty filename submitted.'})
    if file and allowed_file(file.filename):
        filename = file.filename
        print("ficheiro:"+filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        result = process_and_predict(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        print(result)
        response = {'age': result[0], 'gender': result[1]}
        return jsonify(response)
    else:
        return jsonify({'error':'File has invalid extension'})

# Run Service

In [ ]:
if __name__ == '__main__':
    app.run(host= '127.0.0.1',port=5000)